In [1]:
import matplotlib.pyplot as plt
import json
import numpy as np

In [4]:
#///////// INPUTS //////////
distributionFile = "isingBatches/Sampled_TSP_9q_p=1_aer_simulator.json"
exactSolutionsFile = "isingBatches/solved_batch_Ising_data_TSP_9q_.json"

#instanceOfInterest = 1

In [ ]:
def load_file_into_dict(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data



In [ ]:
sampledResults = load_file_into_dict(distributionFile)
exactSolutions = load_file_into_dict(exactSolutionsFile)

qaoaSuccessCount = 0
for instance in sampledResults:
    mostProbableSolution = max(sampledResults[instance], key=sampledResults[instance].get)
    for item in exactSolutions:
        if item["instance_id"] == int(instance):
            correctSolutionsIsing = item["solutions"]
        correctSolutionsBinary = []
    for item in correctSolutionsIsing:
        correctSolutionsBinary.append(item.replace('-1', '1').replace('+1', '0').replace(',', '')) #0s and 1s may seem mixed up here, but its because the qubit state |0> corresponds to the Z-eigenvalue of +1, which is the opposite way round to the QUBO>ising mapping
    if mostProbableSolution in correctSolutionsBinary:
        qaoaSuccessCount += 1
    #print(mostProbableSolution, correctSolutionsBinary, correctSolutionsIsing)
print(f"QAOA Success Count: {qaoaSuccessCount}")

010000111 ['001100010', '010100001'] ['+1,+1,-1,-1,+1,+1,+1,-1,+1', '+1,-1,+1,-1,+1,+1,+1,+1,-1']
011000011 ['100010001', '001010100'] ['-1,+1,+1,+1,-1,+1,+1,+1,-1', '+1,+1,-1,+1,-1,+1,-1,+1,+1']
100000100 ['010001100', '100001010'] ['+1,-1,+1,+1,+1,-1,-1,+1,+1', '-1,+1,+1,+1,+1,-1,+1,-1,+1']
000011000 ['100010001', '001010100'] ['-1,+1,+1,+1,-1,+1,+1,+1,-1', '+1,+1,-1,+1,-1,+1,-1,+1,+1']
010001000 ['100010001', '001010100'] ['-1,+1,+1,+1,-1,+1,+1,+1,-1', '+1,+1,-1,+1,-1,+1,-1,+1,+1']
100000111 ['100010001', '001010100'] ['-1,+1,+1,+1,-1,+1,+1,+1,-1', '+1,+1,-1,+1,-1,+1,-1,+1,+1']
011010011 ['001100010', '010100001'] ['+1,+1,-1,-1,+1,+1,+1,-1,+1', '+1,-1,+1,-1,+1,+1,+1,+1,-1']
010100010 ['001010100', '100010001'] ['+1,+1,-1,+1,-1,+1,-1,+1,+1', '-1,+1,+1,+1,-1,+1,+1,+1,-1']
010111010 ['001010100', '100010001'] ['+1,+1,-1,+1,-1,+1,-1,+1,+1', '-1,+1,+1,+1,-1,+1,+1,+1,-1']
000110000 ['010100001', '001100010'] ['+1,-1,+1,-1,+1,+1,+1,+1,-1', '+1,+1,-1,-1,+1,+1,+1,-1,+1']
001110001 ['01010000